# 11 - Data Preparation with DNS data  - Combined Dataset

This notebook is intended to show the steps required to prepare the data for the ML training.

We use the keras tokenizer to tokenize and onehot encode the dns query information.  Other characteristics of this notebook include:

  * Combined dataset used
  * Geolocation informaiton dropped as this is only reliable for the Fitt dataset




Modified version created on Google drive

In [6]:
from pathlib import Path
import os
import re
import apsw
import pandas as pd
import time

In [7]:
cwd = Path.cwd().parent.parent
print(cwd)

/home/ricdeez/uni/projects/iotnetlearn


In [8]:
dbs = [
    os.path.join(cwd, 'db', 'db01', 'NetCollector.sqlite'),
    os.path.join(cwd, 'db', 'db02', 'NetCollector.sqlite'),
    os.path.join(cwd, 'db', 'db03', 'NetCollector.sqlite'),
]

In [9]:
from sqlalchemy import create_engine
df1 = None
df2 = None
df3 = None
for idx, db_path in enumerate(dbs):
    engine = create_engine(f'sqlite:////{db_path}')

    print(engine)

    sql = """

    select d.srcPort as srcport, 
           d.dstPort as dstport, 
           sum(d.frameSize) as totalframesize, 
           min(d.frameSize) as minframesize,
           max(d.frameSize) as maxframesize,
           avg(d.frameTTL) as framettl,
           dns.dnsquery,
           d.manufacturer,
           d.device_type
    from deviceLog d inner join dnsqueries dns on d.frameNum = dns.pktnum
    where d.device_type is not null
    group by d.srcAddr, d.dstAddr, d.srcPort, d.dstPort

    """

    if idx == 0:
        df1 = pd.read_sql_query(sql, engine)
    elif idx == 1:
        df2 = pd.read_sql_query(sql, engine)
    elif idx == 2:
        df3 = pd.read_sql_query(sql, engine)


Engine(sqlite://///home/ricdeez/uni/projects/iotnetlearn/db/db01/NetCollector.sqlite)
Engine(sqlite://///home/ricdeez/uni/projects/iotnetlearn/db/db02/NetCollector.sqlite)
Engine(sqlite://///home/ricdeez/uni/projects/iotnetlearn/db/db03/NetCollector.sqlite)


In [10]:
df1 = pd.concat([df1, df2], axis='rows', ignore_index=True)
df1 = pd.concat([df1, df3], axis='rows', ignore_index=True)
df = df1
df.describe()


,totalframesize,minframesize,maxframesize,framettl
count,32799.000000,32799.000000,32799.000000,32799.000000
mean,164.587701,84.173207,85.463063,92.856752
std,3267.692046,8.832032,9.917332,48.956212
min,65.000000,65.000000,65.000000,1.000000
25%,81.000000,78.000000,78.000000,64.000000
50%,91.000000,81.000000,81.000000,64.000000
75%,146.000000,91.000000,93.000000,128.000000
max,481258.000000,182.000000,366.000000,255.000000


In [11]:
from keras.preprocessing.text import Tokenizer

def tokenize_and_onehotencode(df, feature_name, num_words=300):
    """
    Augments the dataframe to include a one-hot encoded
    set of columns, limited by the number of words that 
    are passed in by argument
    """
    feature = df[feature_name]    
    tk = Tokenizer()
    tk.fit_on_texts(feature)
    # https://github.com/keras-team/keras/issues/8092
    tk.word_index = {e:i for e,i in tk.word_index.items() if i < num_words} 
    encoded_feature=tk.texts_to_matrix(feature, mode='binary')
    cols = [f'word_idx{i+1}' for i in range(num_words)]
    df2 = pd.DataFrame(data=encoded_feature, columns=cols)
    return df2


Using TensorFlow backend.


In [12]:
df2 = tokenize_and_onehotencode(df, feature_name='dnsquery')
df = pd.concat([df, df2], axis='columns')
df.head()

,srcport,dstport,totalframesize,minframesize,maxframesize,framettl,dnsquery,manufacturer,device_type,word_idx1,...,word_idx291,word_idx292,word_idx293,word_idx294,word_idx295,word_idx296,word_idx297,word_idx298,word_idx299,word_idx300
0,5353,5353,13758,70,82,1.000000,_googlecast._tcp.local.,Rivet Networks,Other,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5353,5353,481258,82,132,255.000000,chromecast-audio-46eabec7bd7a728a79fb6231c4ae5...,"Google, Inc.",Chromecast-Audio,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5353,5353,1891,120,127,255.000000,_00000000-06a1-86ad-4c60-2ffbf090480e._sub._ho...,"Apple, Inc.",Apple TV,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5353,5353,882,104,190,255.000000,_sleep-proxy._udp.local.,"Apple, Inc.",Other,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5353,5353,8755,74,190,250.637931,_sleep-proxy._udp.local.,"Apple, Inc.",Other,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## One-hot encode most common Destination Ports and Source Ports

This uses a simple technique to one-hot encode the most common values for the destination ports and source ports. 

The most common srcport and dstport entries are either 53 or 5353.

In [13]:
import numpy as np

def build_feature_attribute_columns(func, features, vals):
    """
    Augments the dataframe to include a one-hot encoded
    set of columns, limited by the actual vals that are 
    passed in by argument.  

    Also includes a new feature
    for each of the features passed in for the negative
    case where none of the values are matched.
    """
    for feature in features:
        arr = []
        for val in vals:
            func('df[\'{}_is_{}\'] = np.where(df[\'{}\'] == \'{}\', 1, 0)'.format(
                    feature, val, feature, val))
            str = '(df[\'{}\'] != \'{}\')'.format(feature, val)
            arr.append(str)
        where_clause = ' & '.join(arr)
        str = 'df[\'{}_is_other\'] = np.where(({}),1,0)'.format(feature, where_clause)
        func(str)


## Trial run

In [14]:
vals = ['53', '5353']
features = ['dstport', 'srcport']

# Change func to exec to change the dataset
# Warning! this will add features to the current df
build_feature_attribute_columns(func=print,
                                    features=features,
                                    vals=vals)

df['dstport_is_53'] = np.where(df['dstport'] == '53', 1, 0)
df['dstport_is_5353'] = np.where(df['dstport'] == '5353', 1, 0)
df['dstport_is_other'] = np.where(((df['dstport'] != '53') & (df['dstport'] != '5353')),1,0)
df['srcport_is_53'] = np.where(df['srcport'] == '53', 1, 0)
df['srcport_is_5353'] = np.where(df['srcport'] == '5353', 1, 0)
df['srcport_is_other'] = np.where(((df['srcport'] != '53') & (df['srcport'] != '5353')),1,0)


## Final Run

If happy with the output of the above step, we can run the same
function, but this time changing the function that is passed in
to the exec function, causing the dynamically created strings
to be executed and augmenting the df with the new synthetic features.

`Note that the use of the exec feature would make this implementation insecure for any uses other than in an interactive session`

In [15]:
build_feature_attribute_columns(func=exec,
                                    features=features,
                                    vals=vals)
df.head()

,srcport,dstport,totalframesize,minframesize,maxframesize,framettl,dnsquery,manufacturer,device_type,word_idx1,...,word_idx297,word_idx298,word_idx299,word_idx300,dstport_is_53,dstport_is_5353,dstport_is_other,srcport_is_53,srcport_is_5353,srcport_is_other
0,5353,5353,13758,70,82,1.000000,_googlecast._tcp.local.,Rivet Networks,Other,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
1,5353,5353,481258,82,132,255.000000,chromecast-audio-46eabec7bd7a728a79fb6231c4ae5...,"Google, Inc.",Chromecast-Audio,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
2,5353,5353,1891,120,127,255.000000,_00000000-06a1-86ad-4c60-2ffbf090480e._sub._ho...,"Apple, Inc.",Apple TV,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
3,5353,5353,882,104,190,255.000000,_sleep-proxy._udp.local.,"Apple, Inc.",Other,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
4,5353,5353,8755,74,190,250.637931,_sleep-proxy._udp.local.,"Apple, Inc.",Other,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1


In [17]:
drop_columns = ['srcport', 'dstport', 'dnsquery']
df.drop(drop_columns, axis='columns', inplace=True)

## Categorical Data Mapping

For the deep learning algorithm to work, we need to get rid of all categorical data.  For the Manufacturer's we will create a 1:1 mapping of the manufacturer name as per the wireshark OUI lookup dataset and the relative position of that in our ordered array of unique entries

In [18]:
mapping = {k: v for v, k in enumerate((x for x in df['manufacturer'].unique() if len(x)>0),1)}
print(mapping)

df['manufacturer'] = df['manufacturer'].map(mapping)


{'Rivet Networks': 1, 'Google, Inc.': 2, 'Apple, Inc.': 3, 'Technicolor CH USA Inc.': 4, 'Microsoft Corporation': 5, 'Intel Corporate': 6, 'Realtek Semiconductor Corp.': 7, 'Pegatron Corporation': 8, 'Ubiquiti Networks Inc.': 9, 'Sony Corporation': 10, 'ASUSTek COMPUTER INC.': 11, 'Microsoft': 12, 'Samsung Electro-Mechanics(Thailand)': 13, 'Netatmo': 14, 'Lifi Labs Management Pty Ltd': 15, 'Invoxia': 16, 'Withings': 17, 'Shenzhen Reecam Tech.Ltd.': 18, 'Belkin International Inc.': 19, 'Physical Graph Corporation': 20, 'AzureWave Technology Inc.': 21, 'Samsung Electronics Co.,Ltd': 22, 'Amazon Technologies Inc.': 23, 'HTC Corporation': 24}


## Min-max scaling of values

The following code performs min-max scaling of our numerical features:

\begin{align}
\mathcal{Z} & = \frac{x\tiny{i} \small- min(X)}{max(X) - min(X)} 
\end{align}


In [19]:
features = ['totalframesize', 'minframesize', 'maxframesize', 'framettl']
for feature in features:
    df[feature] = ((df[feature] - df[feature].min()) / 
        (df[feature].max() - df[feature].min()))

## Take a random sample or not

The following code allows us to check our model on a small subset
of the data.  In this case, with frac set to 0.1 we are randomly sampling 10% of the data

In [20]:
# Comment one of the lines below accordingly
# ------------------------------------------

# df_sample = df.sample(frac=0.1)
df_sample = df
df_sample.describe()

,totalframesize,minframesize,maxframesize,framettl,manufacturer,word_idx1,word_idx2,word_idx3,word_idx4,word_idx5,...,word_idx297,word_idx298,word_idx299,word_idx300,dstport_is_53,dstport_is_5353,dstport_is_other,srcport_is_53,srcport_is_5353,srcport_is_other
count,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.0,32799.000000,32799.000000,32799.000000,32799.000000,...,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000
mean,0.000207,0.163874,0.067984,0.361641,9.795299,0.0,0.848532,0.361779,0.203817,0.203787,...,0.001220,0.001220,0.001220,0.001220,0.830574,0.000335,0.169091,0.000030,0.000366,0.999604
std,0.006791,0.075487,0.032948,0.192741,3.263756,0.0,0.358510,0.480523,0.402841,0.402818,...,0.034901,0.034901,0.034901,0.034901,0.375133,0.018310,0.374838,0.005522,0.019124,0.019905
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000033,0.111111,0.043189,0.248031,9.000000,0.0,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000054,0.136752,0.053156,0.248031,9.000000,0.0,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000168,0.222222,0.093023,0.500000,9.000000,0.0,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,24.000000,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Save  df_sample DataFrame for future use

In order to allow us to commence our training without having to repeat these steps, we will now save the df_sample dataframe using the pandas built-in to_json method.

In [21]:
import os
from pathlib import Path
cwd = Path.cwd()

file_path = os.path.join(cwd, 'saved_datasets', 'combined_dataset_dns.json' )
df_sample.to_json(file_path)
print("[+] Stored pandas dataframe to disk")

[+] Stored pandas dataframe to disk


In [25]:
df_sample.head()

,totalframesize,minframesize,maxframesize,framettl,manufacturer,device_type,word_idx1,word_idx2,word_idx3,word_idx4,...,word_idx297,word_idx298,word_idx299,word_idx300,dstport_is_53,dstport_is_5353,dstport_is_other,srcport_is_53,srcport_is_5353,srcport_is_other
0,0.028456,0.042735,0.056478,0.000000,1,Other,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
1,1.000000,0.145299,0.222591,1.000000,2,Chromecast-Audio,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
2,0.003795,0.470085,0.205980,1.000000,3,Apple TV,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
3,0.001698,0.333333,0.415282,1.000000,3,Other,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1
4,0.018059,0.076923,0.415282,0.982827,3,Other,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,0,0,1


In [26]:
df_sample.shape

(32799, 312)